## Import

In [1]:
import pandas as pd 
from datetime import datetime  
import numpy as np 
import os 
import sys
import torch

# Personnal Import 
from utilities_DL import get_loss,choose_optimizer,load_model,load_all
from DL_class import Trainer,PI_object
from PI import plot_bands_CQR
from config import get_config,get_parameters
# ...

# Paths
folder_path = 'data/'
file_name = 'preprocessed_subway_15_min.csv'

## Load Raw Data and forbidden dates : 

In [2]:
#Init and load data: 
time_step_per_hour=4
H,W,D = 6,1,1
L = H+W+D 
step_ahead = 1

window_pred = np.arange(2*96)

# Load subway in data:
subway_in = pd.read_csv(folder_path+file_name,index_col = 0)
subway_in.columns.name = 'Station'
subway_in.index = pd.to_datetime(subway_in.index)

# Invalid dates : 
invalid_dates = pd.date_range(datetime(2019,4,23,14),datetime(2019,4,28,14),freq = f'{60/time_step_per_hour}min')

## Get Parameters

In [3]:
# Choose Model :
model_name = 'CNN'#'STGCN' 
single_station = False   # Quick Training if True

# Choose config 
config = get_config(model_name = model_name,other_params= {'seq_length':L})
#config = get_config(model_name = model_name,learn_graph_structure = True,other_params= {'seq_length':L})  # MTGNN
args = get_parameters(config)

# Modification : 
args.epochs = 2
args.optimizer = 'adamw'

if model_name == 'STGCN':
    args.gso_type = 'sym_norm_lap'
    args.graph_conv_type = 'graph_conv'
    args.act_fun = 'glu'
    args.Ks = 2



## Training sans embedding : 

In [ ]:
from bokeh.plotting import output_file, save, showt
from plotting_bokeh import plot_prediction

# ======== No specific LR cause no Embedding ========
args.specific_lr = False
# ========================================================

dataset,data_loader,dic_class2rpz,dic_rpz2class,args_embedding,loss_function,model,optimizer = load_all(subway_in,args,time_step_per_hour,step_ahead,H,D,W,invalid_dates,
            embedding_dim=2,position = 'input',single_station = single_station)

# Pas d'embedding : 
model = load_model(args,None).to(args.device)
optimizer = choose_optimizer(model,args)

# Metrics and save :
metrics = ['Epochs','train_loss','valid_loss','PICP','MPIW','PICP_cqr','MPIW_cqr']
results = pd.DataFrame(columns = metrics)

if args.model_name == 'STGCN':
    save_dir = f"save/{args.model_name}/{args.graph_conv_type}/{args.gso_type}/act_{args.act_func}/Ks{args.Ks}/opt_{args.optimizer}/"
if args.model_name == 'CNN':
    save_dir = f"save/{args.model_name}/h_dims{'_'.join(list(map(str,args.H_dims)))}/out_dims{'_'.join(list(map(str,args.C_outs)))}/opt_{args.optimizer}/"  
    
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
best_model_save =  f'{save_dir}best_model_without_embedding.pkl'

trial_save_init = f""

trainer = Trainer(model,data_loader,args,optimizer,loss_function,scheduler = None,args_embedding = None,save_path =best_model_save)

# Sauvegarde les meilleurs modèle sur lesquels on pourra après coup faire différentes calibrations.
trainer.train_and_valid(mod = 1000)  # Récupère les conformity scores sur I1, avec les estimations faites precedemment 

# Si on veut juste les Visus du meilleur model : 
saved_checkpoint = torch.load(best_model_save)
trainer.model.load_state_dict(saved_checkpoint['state_dict'])
# Calib
Q = trainer.conformal_calibration(args.alpha,dataset,conformity_scores_type =args.conformity_scores_type, quantile_method = args.quantile_method)
# Plotting, results, and save it in .html file 
best_epoch = saved_checkpoint['epoch']
if args.model_name == 'STGCN':
    trial_save = f"Epochs{best_epoch}_best_model_without_embedding"

pi,pi_cqr,p1 = plot_prediction(trainer,dataset,Q,args,station = 0)
save_path = f"Epochs{best_epoch}_best_model_without_embedding"
output_file(f"{save_dir}{save_path}.html")
save(p1)
show(p1)


results.loc[len(results)] = dict(Epochs = best_epoch, train_loss = trainer.train_loss[best_epoch], valid_loss = trainer.valid_loss[best_epoch], PICP = pi.picp, MPIW = pi.mpiw, PICP_cqr = pi_cqr.picp, MPIW_cqr = pi_cqr.mpiw)
results.to_csv(f"{save_dir}{save_path}.csv")           ## Plot Latent Space Bokeh 

## Plot Latent Space Bokeh 

In [4]:
from plotting_bokeh import  generate_bokeh

In [7]:
for calendar_class in [0,1,2,3]:
    for bool_specific_lr in [True,False]:
        for position in ['input','output']:
            # Update args 
            args.calendar_class = calendar_class
            args.specific_lr = bool_specific_lr
            # ...
    
            dataset,data_loader,dic_class2rpz,dic_rpz2class,args_embedding,loss_function,model,optimizer = load_all(subway_in,args,time_step_per_hour,step_ahead,H,D,W,invalid_dates,
                        embedding_dim=2,position = position,single_station = single_station)
            # Metrics and save :
            metrics = ['Epochs','train_loss','valid_loss','PICP','MPIW','PICP_cqr','MPIW_cqr']
            results = pd.DataFrame(columns = metrics)
            # Save Directory
            if args.model_name == 'STGCN':
                save_dir = f"save/{args.model_name}/{args.graph_conv_type}/{args.gso_type}/act_{args.act_func}/Ks{args.Ks}/Specific_lr_{args.specific_lr}/CalendarClass{args.calendar_class}/position_{args_embedding.position}/opt_{args.optimizer}/"
            if args.model_name == 'CNN':
                save_dir = f"save/{args.model_name}/h_dims{'_'.join(list(map(str,args.H_dims)))}/out_dims{'_'.join(list(map(str,args.C_outs)))}/Specific_lr_{args.specific_lr}/CalendarClass{args.calendar_class}/position_{args_embedding.position}/opt_{args.optimizer}/"  
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            best_model_save =  f'{save_dir}best_model.pkl'

            trial_save_init = f""

            # Trainer : 
            trainer = Trainer(model,data_loader,args,optimizer,loss_function,scheduler = None,args_embedding = args_embedding,save_path =best_model_save)  # Ajoute dans trainer, if calibration_prop is not None .... et on modifie le dataloader en ajoutant un clabration set
            

            # =========
            if args.calendar_class == 0:
                assert trainer.model.Tembedding.embedding.in_features == 1, f'trainer.model.Tembedding.embedding.in_features = {trainer.model.Tembedding.embedding.in_features} instead of {1} cause calendar class is {0}'

            if args.calendar_class == 1:
                assert trainer.model.Tembedding.embedding.in_features == 50, f'trainer.model.Tembedding.embedding.in_features = {trainer.model.Tembedding.embedding.in_features} instead of {50} cause calendar class is {args.calendar_class}'

            if args.calendar_class == 2:
                assert trainer.model.Tembedding.embedding.in_features == 4*24*7, f'trainer.model.Tembedding.embedding.in_features = {trainer.model.Tembedding.embedding.in_features} instead of {4*24*7} cause calendar class is {args.calendar_class}'

            if args.calendar_class == 3:
                assert trainer.model.Tembedding.embedding.in_features == 24*7, f'trainer.model.Tembedding.embedding.in_features = {trainer.model.Tembedding.embedding.in_features} instead of {4*24} cause calendar class is {args.calendar_class}'
            # =========

            # Sauvegarde les meilleurs modèle sur lesquels on pourra après coup faire différentes calibrations.
            trainer.train_and_valid(mod = 1000)  # Récupère les conformity scores sur I1, avec les estimations faites precedemment 

            # Si on veut voir la dispersion / l'arrangement des embedding temporel au cours du temps :
            if False : 
                for nb_visu in range(1):
                    # Training, Calib
                    trainer.train_and_valid(mod = 1000)  # Récupère les conformity scores sur I1, avec les estimations faites precedemment 
                    Q = trainer.conformal_calibration(args.alpha,dataset,conformity_scores_type =args.conformity_scores_type, quantile_method = args.quantile_method)
                    # ...

                    # Save and plotting 
                    current_epoch = (nb_visu+1)*args.epochs
                    if args.model_name == 'STGCN':
                        trial_save = f"Epochs{current_epoch}{trial_save_init}"
                    pi,pi_cqr = generate_bokeh(trainer,data_loader,dataset,Q,args,dic_class2rpz,save_dir,trial_save,station = 0)

                    results.loc[len(results)] = dict(Epochs = current_epoch, train_loss = trainer.train_loss[-1], valid_loss = trainer.valid_loss[-1], PICP = pi.picp, MPIW = pi.mpiw, PICP_cqr = pi_cqr.picp, MPIW_cqr = pi_cqr.mpiw)

                results.to_csv(f"{save_dir}{trial_save}.csv")

            # Si on veut juste les Visus du meilleur model : 
            saved_checkpoint = torch.load(best_model_save)
            trainer.model.load_state_dict(saved_checkpoint['state_dict'])
            # Calib
            Q = trainer.conformal_calibration(args.alpha,dataset,conformity_scores_type =args.conformity_scores_type, quantile_method = args.quantile_method)
            # Plotting, results, and save it in .html file 
            best_epoch = saved_checkpoint['epoch']
            if args.model_name == 'STGCN':
                trial_save = f"Epochs{best_epoch}_best_model"
            pi,pi_cqr = generate_bokeh(trainer,data_loader,dataset,Q,args,dic_class2rpz,save_dir,trial_save,station = 0)

            results.loc[len(results)] = dict(Epochs = best_epoch, train_loss = trainer.train_loss[best_epoch], valid_loss = trainer.valid_loss[best_epoch], PICP = pi.picp, MPIW = pi.mpiw, PICP_cqr = pi_cqr.picp, MPIW_cqr = pi_cqr.mpiw)
            results.to_csv(f"{save_dir}{trial_save}.csv")           



40 nodes (stations) have been considered. 
 
Model : STGCN 
 Optimizer: adamw 
 A specific LR by layer is used 
 Calendar class: 0 
 Quantile Method: weekday_hour 
 Encoding dimension: 1. Is related to Dictionnary size of the Temporal Embedding Layer 
 Embedding dimension: 2 
 Position of the Embedding layer: input 
Ko: 8, enable padding: True
start training
epoch: 0 
 min\epoch : 0.22
Estimated time for training: 0.4min 
Proportion of label with quantile order set to 1: 0.0%


/Users/romainrochas/opt/anaconda3/envs/preprocessingclone/lib/python3.7/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([1133, 40, 1])) that is different to the input size (torch.Size([1133, 40, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/romainrochas/opt/anaconda3/envs/preprocessingclone/lib/python3.7/site-packages/torch/nn/modules/loss.py:94: UserWarning: Using a target size (torch.Size([1133, 40, 1])) that is different to the input size (torch.Size([1133, 40, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Loading BokehJS ...

40 nodes (stations) have been considered. 
 
Model : STGCN 
 Optimizer: adamw 
 A specific LR by layer is used 
 Calendar class: 0 
 Quantile Method: weekday_hour 
 Encoding dimension: 1. Is related to Dictionnary size of the Temporal Embedding Layer 
 Embedding dimension: 2 
 Position of the Embedding layer: output 
Ko: 8, enable padding: True
start training
epoch: 0 
 min\epoch : 0.18
Estimated time for training: 0.4min 
Proportion of label with quantile order set to 1: 0.0%


/Users/romainrochas/opt/anaconda3/envs/preprocessingclone/lib/python3.7/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([1133, 40, 1])) that is different to the input size (torch.Size([1133, 40, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/romainrochas/opt/anaconda3/envs/preprocessingclone/lib/python3.7/site-packages/torch/nn/modules/loss.py:94: UserWarning: Using a target size (torch.Size([1133, 40, 1])) that is different to the input size (torch.Size([1133, 40, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


...

/Users/romainrochas/opt/anaconda3/envs/preprocessingclone/lib/python3.7/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([1133, 40, 1])) that is different to the input size (torch.Size([1133, 40, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/romainrochas/opt/anaconda3/envs/preprocessingclone/lib/python3.7/site-packages/torch/nn/modules/loss.py:94: UserWarning: Using a target size (torch.Size([1133, 40, 1])) that is different to the input size (torch.Size([1133, 40, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Loading BokehJS ...

40 nodes (stations) have been considered. 
 
Model : STGCN 
 Optimizer: adamw 
 The same LR is used for each layer 
 Calendar class: 3 
 Quantile Method: weekday_hour 
 Encoding dimension: 168. Is related to Dictionnary size of the Temporal Embedding Layer 
 Embedding dimension: 2 
 Position of the Embedding layer: output 
Ko: 8, enable padding: True
start training
epoch: 0 
 min\epoch : 0.18
Estimated time for training: 0.4min 
Proportion of label with quantile order set to 1: 100.0%


/Users/romainrochas/opt/anaconda3/envs/preprocessingclone/lib/python3.7/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([1133, 40, 1])) that is different to the input size (torch.Size([1133, 40, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/romainrochas/opt/anaconda3/envs/preprocessingclone/lib/python3.7/site-packages/torch/nn/modules/loss.py:94: UserWarning: Using a target size (torch.Size([1133, 40, 1])) that is different to the input size (torch.Size([1133, 40, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Loading BokehJS ...

In [ ]:
config = get_config(model_name = 'CNN')

config['conformity_scores_type'] = 'max_residual' 
config['seq_length'] = L

args = get_parameters(config)
args.epochs = 100

# Load train and calib model 
print(f"Type of model: {args.model_name} \n Loss function type: {args.loss_function_type} \n Quantile Method : {args.quantile_method} \n Calendar Class: {args.calendar_class} \n")
(dataset,U,Utarget) = load_normalized_dataset(subway_in,time_step_per_hour,args.train_prop,step_ahead,H,D,W,invalid_dates)
time_slots_labels,dic_class2rpz,dic_rpz2class = get_time_slots_labels(dataset,type_class= args.calendar_class)
data_loader_obj = DictDataLoader(U,Utarget,args.train_prop,args.valid_prop,validation = 'classic', shuffle = True, calib_prop=args.calib_prop, time_slots = time_slots_labels)
data_loader = data_loader_obj.get_dictdataloader(args.batch_size)

# Quantile Loss
quantiles = torch.Tensor([args.alpha/2,1-args.alpha/2]).to(args.device)
assert args.out_dim == len(quantiles), "Output dimension doesn't match with the number of estimated quantiles"
loss_function = get_loss(args.loss_function_type,quantiles)

# Load model : 
model = load_model(args,args_embedding)
model.to(args.device)
optimizer = choose_optimizer(model,args)

trainer = Trainer(model,data_loader,args,optimizer,loss_function,scheduler = None,args_embedding = args_embedding)  # Ajoute dans trainer, if calibration_prop is not None .... et on modifie le dataloader en ajoutant un clabration set
trainer.train_and_valid(mod = 1000)  # Récupère les conformity scores sur I1, avec les estimations faites precedemment 
Q = trainer.conformal_calibration(args.alpha,dataset,conformity_scores_type =args.conformity_scores_type, quantile_method = args.quantile_method)  # calibration for PI 90%
    
(preds,Y_true,T_labels,df_metrics) = trainer.testing(dataset,metrics= ['mse','mae'])
# ...


pi = trainer.CQR_PI(preds,Y_true,args.alpha,Q,T_labels.long())
results = get_dic_results(trainer,pi)

conformity_scores = trainer.conformity_scores
plot_bands_CQR(trainer,Y_true,preds,pi,window_pred,args.alpha,conformity_scores,results,bins = 100)

## Load Model 

In [ ]:
nb_trial = 1

for _ in range(nb_trial):
    config = get_config(model_name = 'CNN')

    config['conformity_scores_type'] = 'max_residual' 
    config['seq_length'] = L

    args = get_parameters(config)
    args.epochs = 100

    dataset,trainer,Q,preds,Y_true,T_labels = load_train_calib_model(args,args_embedding)
    pi = trainer.CQR_PI(preds,Y_true,args.alpha,Q,T_labels.long())
    results = get_dic_results(trainer,pi)

    conformity_scores = trainer.conformity_scores
    plot_bands_CQR(trainer,Y_true,preds,pi,window_pred,args.alpha,conformity_scores,results,bins = 100)

## Visu pour clustering 

In [ ]:
T_labels  # T_labels issus du Training 
trainer.model.eval()   # pas grad, pas de dropout 
with torch.no_grad():
    output = trainer.model.Tembedding(T_labels.long())

X1,Y1,Z1 = output[:,0].numpy(),output[:,1].numpy(),output[:,2].numpy()

x = np.arange(len(X1))
ax = plt.figure().add_subplot(projection='3d')
ax.scatter(X1,Y1,Z1,label = 'embedding')
ax.legend()
plt.show()

In [ ]:
for training_mode in ['cal','train','validate','test']:
    print(training_mode)
    Pred_cal,Y_true_cal =trainer.test_prediction(allow_dropout = False,training_mode = 'cal')
    unorm_Pred_cal,unorm_Y_true_cal = dataset.unormalize_tensor(Pred_cal),dataset.unormalize_tensor(Y_true_cal)

    # PI 'classic' :
    pi_cal = PI_object(unorm_Pred_cal,unorm_Y_true_cal,alpha = args.alpha, type_calib = 'classic')
    print(pi_cal.mpiw,pi_cal.picp)

    # PI 'CQR' : 
    pi_cqr_cal = PI_object(unorm_Pred_cal,unorm_Y_true_cal,alpha = args.alpha, Q = Q, type_calib = 'CQR')
    print(pi_cqr_cal.mpiw,pi_cqr_cal.picp)

    plt.plot(np.arange(100),pi_cqr_cal.upper[:100,0,0],color = 'green',linestyle = 'dashed',label = f"PI, with Q = {'{:.2f}'.format(pi_cqr_cal.Q[0,0,0].item())}")
    plt.plot(np.arange(100),pi_cqr_cal.lower[:100,0,0],color = 'green',linestyle = 'dashed')
    plt.plot(np.arange(100),pi_cal.upper[:100,0,0],color = 'red',linestyle = 'dashed',label = 'quantile')
    plt.plot(np.arange(100),pi_cal.lower[:100,0,0],color = 'red',linestyle = 'dashed')
    plt.plot(np.arange(100),unorm_Y_true_cal[:100,0,0],color = 'blue')
    plt.legend()

In [ ]:
# PI des quantiles est trop large (97%). Donc, Q devrait être négatif 
model = trainer.model
model.eval()
with torch.no_grad():
    data = [[x_b,y_b] for  x_b,y_b in trainer.dataloader['cal']]
    X_cal,Y_cal = torch.cat([x_b for [x_b,y_b] in data]),torch.cat([y_b for [x_b,y_b] in data])
    preds = model(X_cal) # x_cal is normalized

    # get lower and upper band
    if preds.size(-1) == 2:
        lower_q,upper_q = preds[...,0].unsqueeze(-1),preds[...,1].unsqueeze(-1)   # The Model return ^q_l and ^q_u associated to x_b

    elif preds.size(-1) == 1:
        lower_q,upper_q = preds,preds 
    else:
        raise ValueError(f"Shape of model's prediction: {preds.size()}. Last dimension should be 1 or 2.")
    
    # unormalized lower band, upper band, and Y_cal 
    lower_q, upper_q = dataset.unormalize_tensor(lower_q),dataset.unormalize_tensor(upper_q)
    Y_cal = dataset.unormalize_tensor(Y_cal)

    # Confority scores and quantiles
    if args.conformity_scores_type == 'max_residual':
        conformity_scores = torch.max(lower_q-Y_cal,Y_cal-upper_q) # Element-wise maximum        #'max(lower_q-y_b,y_b-upper_q)' is the quantile regression error function
    if args.conformity_scores_type == 'max_residual_plus_middle':
        print("|!| Conformity scores computation is not based on 'max(ql-y, y-qu)'")
        conformity_scores = torch.max(lower_q-Y_cal,Y_cal-upper_q) + ((lower_q>Y_cal)(upper_q<Y_cal))*(upper_q - lower_q)/2  # Element-wise maximum        #'max(lower_q-y_b,y_b-upper_q)' is the quantile regression error function 

    quantile_order = torch.Tensor([np.ceil((1 - args.alpha)*(X_cal.size(0)+1))/X_cal.size(0)])
    Q = torch.quantile(conformity_scores, quantile_order, dim = 0) #interpolation = 'higher'

In [ ]:
lower_q[:10,0,0],upper_q[:10,0,0],Y_cal[:10,0,0]

In [ ]:
Pred_test,Y_true_test =trainer.test_prediction(allow_dropout = False,training_mode = 'test')
pi_test = PI_object(Pred_test,Y_true_test,alpha = args.alpha, type_calib = 'classic')
print(pi_test.mpiw,pi_test.picp)

plt.plot(np.arange(100),Pred_test[:100,0,0],color = 'red',linestyle = 'dashed')
plt.plot(np.arange(100),Pred_test[:100,0,1],color = 'red',linestyle = 'dashed')
plt.plot(np.arange(100),Y_true_test[:100,0,0],color = 'blue')

In [ ]:
Pred_valid,Y_true_valid =trainer.test_prediction(allow_dropout = False,training_mode = 'validate')
pi_valid = PI_object(Pred_valid,Y_true_valid,alpha = args.alpha, type_calib = 'classic')
print(pi_valid.mpiw,pi_valid.picp)

plt.plot(np.arange(100),Pred_valid[:100,0,0],color = 'red',linestyle = 'dashed')
plt.plot(np.arange(100),Pred_valid[:100,0,1],color = 'red',linestyle = 'dashed')
plt.plot(np.arange(100),Y_true_valid[:100,0,0],color = 'blue')

In [ ]:
Pred_train,Y_true_train =trainer.test_prediction(allow_dropout = False,training_mode = 'train')
pi_train = PI_object(Pred_train,Y_true_train,alpha = args.alpha, type_calib = 'classic')
print(pi_train.mpiw,pi_train.picp)

plt.plot(np.arange(100),Pred_train[:100,0,0],color = 'red',linestyle = 'dashed')
plt.plot(np.arange(100),Pred_train[:100,0,1],color = 'red',linestyle = 'dashed')
plt.plot(np.arange(100),Y_true_train[:100,0,0],color = 'blue')

A priori, ça ne sert à rien de les projeté dans un espace de plus grande dimension.
- J'ai un ensemble de 'mot'. Chaque mot correspond a une combinaison (d,h,m).
- Il y a 7 jours, 24h, 4 time-step de minute (0,15,30,45). Donc j'ai un ensemble de 4*7*24 = 672 mots.
- J'aimerais faire un embedding de ces mots. C'est a dire représenter chaque mot par un vecteur. 

Exemple : 
(0,7,0) est 'Lundi 8h15'. J'en fais un embedding (donc une projection) dans un espace latent (exemple en dimension 3):


**Problème** : très peu d'occurence de chacun des 'mots' (En tout une semaine pour balayer une seule fois chaucn des mots possible. Donc 1 an de Training c'est seulement 57 apparitions...)